In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import re
import csv

TEST CELL - Version 1 - scrap anything

In [10]:
# options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # Run without opening a browser
# options.add_argument("start-maximized")
# options.add_argument("--disable-blink-features=AutomationControlled")  # Reduce bot detection
# options.add_argument("user-agent=Mozilla/5.0")  # Mimic a real browser

# # Initialize the driver
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# # Open the website
# URL = "https://www.fragrantica.com/search/"  # Replace with the actual page containing the perfumes
# driver.get(URL)

# # Allow time for JavaScript to load
# driver.implicitly_wait(3)

# # Extract elements by class name
# items = driver.find_elements(By.CLASS_NAME, "cell.card.fr-news-box")

# # Print perfume names (if they have a sub-element with text)
# for item in items:
#     print(item.text)
#     link = item.find_element(By.TAG_NAME, "a").get_attribute("href")
#     print(link)
#     print('\n')
    
# # Close driver
# driver.quit()

TEST CELL - Version 2 - scrap perfume

In [2]:
# Find accords
def find_accords(driver):
    accord_list = []
    accords = driver.find_elements(By.CLASS_NAME, "accord-bar")
    for accord in accords:
        style_attribute = accord.get_attribute("style")
        match = re.search(r'width:\s*([\d.]+%)', style_attribute)
        width_value = match.group(1) if match else "N/A"
        accord_list.append((accord.text, width_value))
    return accord_list
    
# Find description   
def find_description(driver):
    return driver.find_element(By.XPATH,"//div[contains(@itemprop, 'description')]").text

# Find pros and cons
def find_pros_and_cons(driver):
    pros_and_cons = []
    try:
        pros = driver.find_elements(By.CLASS_NAME, "cell.small-12")
        for pro in pros:
            style = pro.get_attribute("style")  # Get the style attribute
            if "display: inline-flex;" in style and "margin: 0.4rem;" in style and "overflow-wrap: break-word;" in style and "padding: 0px 0.6rem;" in style:
                spans = pro.find_elements(By.TAG_NAME, "span")  # Get all <span> elements inside the div
                if spans:
                    last_span_text = spans[-1].get_attribute("textContent")  # Get the last span
                    pros_and_cons.append(last_span_text)
        return pros_and_cons
    except:
        return None
            
# Find notes
def find_notes(driver):
    pyramid_element = driver.find_element(By.XPATH, '//*[@id="pyramid" and contains(@class, "grid-x") and contains(@class, "grid-padding-y")]')
    ingredients_html = pyramid_element.get_attribute('outerHTML')
    soup = BeautifulSoup(ingredients_html, 'html.parser')

    # Initialize the three lists
    top_notes = []
    middle_notes = []
    base_notes = []

    # Find all h4 tags that contain the note categories
    h4_tags = soup.find_all('h4')
    if h4_tags:
        for h4 in h4_tags:
            if 'Top Notes' in h4.text:
                notes_div = h4.find_next_sibling('div')
                top_notes = [div.text.strip() for div in notes_div.find_all('div') 
                            if div.find('a') and not div.find('img')]
            elif 'Middle Notes' in h4.text:
                notes_div = h4.find_next_sibling('div')
                middle_notes = [div.text.strip() for div in notes_div.find_all('div') 
                            if div.find('a') and not div.find('img')]
            elif 'Base Notes' in h4.text:
                notes_div = h4.find_next_sibling('div')
                base_notes = [div.text.strip() for div in notes_div.find_all('div') 
                            if div.find('a') and not div.find('img')]
    else:
        note_divs = soup.find_all('div', style=lambda value: value and 'margin: 0.2rem; display: flex; justify-content: center; flex-direction: column; text-align: center' in value)
        top_notes = [div.find('div').find_next_sibling('div').get_text(strip=True) for div in note_divs]

    return top_notes, middle_notes, base_notes

def find_gender(driver):
    gender = driver.find_element(By.XPATH,"//div[contains(@id, 'toptop')]").text.split('for ')[2]
    return gender

def find_longevity(driver):
    longevity = driver.find_elements(By.CLASS_NAME, "cell.small-12.medium-6")[4].text
    lines = longevity.split('\n')[2:]
    result = "; ".join(f"{lines[i]}: {lines[i+1]}" for i in range(0, len(lines), 2))
    return result

def find_sillage(driver):
    # all_elements= driver.find_elements(By.CLASS_NAME, "cell.small-12.medium-6")
    # sillage = all_elements[5].text
    # lines = sillage.split('\n')[2:]
    # result = "; ".join(f"{lines[i]}: {lines[i+1]}" for i in range(0, len(lines), 2))
    sillage = driver.find_elements(By.CLASS_NAME, "cell.small-12.medium-6")[5].text
    lines = sillage.split('\n')[2:]
    result = "; ".join(f"{lines[i]}: {lines[i+1]}" for i in range(0, len(lines), 2))
    return result
    
def find_price(driver):
    price = driver.find_elements(By.CLASS_NAME, "cell.small-12.medium-6")[7].text
    lines = price.split('\n')[2:]
    result = "; ".join(f"{lines[i]}: {lines[i+1]}" for i in range(0, len(lines), 2))
    return result

# Call all scrapping functions
def scrap_perfume(driver, url):
    driver.get(url)
    driver.implicitly_wait(5)
    
    # print(find_accords(driver))
    # print(find_description(driver))
    # print(find_pros_and_cons(driver))
    print(find_notes(driver))
    # print(find_gender(driver))
    # print(find_longevity(driver))
    # print(find_price(driver))
    
# options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # Run without opening a browser
# options.add_argument("start-maximized")
# options.add_argument("--disable-blink-features=AutomationControlled")  # Reduce bot detection
# options.add_argument("user-agent=Mozilla/5.0")  # Mimic a real browser

# # Initialize the driver
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# scrap_perfume(driver, 'https://www.fragrantica.com/perfume/Lattafa-Perfumes/Khamrah-75805.html')

TEST CELL

In [35]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run without opening a browser
options.add_argument("start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")  # Reduce bot detection
options.add_argument("user-agent=Mozilla/5.0")  # Mimic a real browser
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
scrap_perfume(driver, 'https://www.fragrantica.com/perfume/The-Dua-Brand/Couture-Crumbled-93882.html')
# driver.implicitly_wait(5)

# price = driver.find_elements(By.CLASS_NAME, "cell.small-12.medium-6")[7].text
# lines = price.split('\n')[2:]
# result = "; ".join(f"{lines[i]}: {lines[i+1]}" for i in range(0, len(lines), 2))
# print(result)

# price = driver.find_elements(By.CLASS_NAME, "cell.small-12.medium-6")
# for index, element in enumerate(price):
#     print(index, element.text)

(['Croissant', 'Sandalwood', 'Vanilla', 'Tonka Bean', 'Wild Berries', 'Black Currant'], [], [])


WORKING CODE !!! Version 3 - class to scrap list and perfume

In [2]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from bs4 import BeautifulSoup

class Driver:
    def __init__(self):
        self.options = uc.ChromeOptions()
        self.options.headless = False  # Keep headless mode OFF to avoid detection
        self.options.add_argument("--disable-blink-features=AutomationControlled")
        self.options.add_argument("user-agent=Mozilla/5.0")  # Custom user agent

        self.driver = uc.Chrome(options=self.options)
        self.wait = WebDriverWait(self.driver, 10)
        
    def scrap_list(self, url, how_many_reloads):
        self.driver.get(url)
        self.click_show_more(how_many_reloads)
        perfumes_list = []
        perfumes = self.wait.until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "cell.card.fr-news-box"))
        )
        
        for perfume in perfumes:
            try:
                link = self.wait.until(
                    lambda d: perfume.find_element(By.TAG_NAME, "a")
                ).get_attribute("href")
                name = perfume.text.split('\n')
                perfumes_list.append((name, link))
            except Exception as e:
                print(f"Error processing perfume: {e}")
                continue
                
        return perfumes_list
    
    def find_gender(self):
        try:
            gender_text = self.wait.until(
                EC.presence_of_element_located((By.XPATH, "//div[contains(@id, 'toptop')]"))
            ).text
            return gender_text.split('for ')[1] if 'for ' in gender_text else "Unknown"
        except:
            return "Unknown"
    
    def find_accords(self):
        accord_list = []
        try:
            accords = self.wait.until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, "accord-bar"))
            )
            for accord in accords:
                style_attribute = accord.get_attribute("style")
                match = re.search(r'width:\s*([\d.]+%)', style_attribute)
                width_value = match.group(1) if match else "N/A"
                accord_list.append((accord.text, width_value))
        except Exception as e:
            print(f"Error finding accords: {e}")
        return accord_list
        
    def find_description(self):
        try:
            description = self.wait.until(
                EC.presence_of_element_located((By.XPATH, "//div[@itemprop='description']"))
            )
            try: 
                description = description.text.replace("\n", "")
                return description
            except:
                return None
        except Exception as e:
            print(f"Error finding description: {e}")
            return None

    def find_pros_and_cons(self):
        pros_and_cons = []
        try:
            pros = self.wait.until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, "cell.small-12"))
            )
            for pro in pros:
                try:
                    style = pro.get_attribute("style") or ""
                    if all(x in style for x in ["display: inline-flex;", "margin: 0.4rem;", 
                                              "overflow-wrap: break-word;", "padding: 0px 0.6rem;"]):
                        spans = pro.find_elements(By.TAG_NAME, "span")
                        if spans:
                            last_span_text = spans[-1].get_attribute("textContent")
                            pros_and_cons.append(last_span_text.strip())
                except Exception as e:
                    print(f"Error processing pro/con: {e}")
                    continue
        except Exception as e:
            print(f"Error finding pros/cons: {e}")
        return pros_and_cons if pros_and_cons else None
                
    def find_notes(self):
        try:
            pyramid_element = self.wait.until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="pyramid"]'))
            )
            ingredients_html = pyramid_element.get_attribute('outerHTML')
            soup = BeautifulSoup(ingredients_html, 'html.parser')

            notes = {"top": [], "middle": [], "base": []}
            
            h4_notes = soup.find_all('h4')
            if h4_notes:
                for h4 in h4_notes:
                    notes_div = h4.find_next_sibling('div')
                    if not notes_div:
                        continue
                        
                    note_divs = [div for div in notes_div.find_all('div') 
                                if div.find('a') and not div.find('img')]
                    note_names = [div.get_text(strip=True) for div in note_divs]
                    
                    if 'Top Notes' in h4.text:
                        notes["top"] = note_names
                    elif 'Middle Notes' in h4.text:
                        notes["middle"] = note_names
                    elif 'Base Notes' in h4.text:
                        notes["base"] = note_names                        
            else:
                note_divs = soup.find_all('div', style=lambda value: value and 'margin: 0.2rem; display: flex; justify-content: center; flex-direction: column; text-align: center' in value)
                notes["top"] = [div.find('div').find_next_sibling('div').get_text(strip=True) for div in note_divs]

            return notes["top"], notes["middle"], notes["base"]
        except Exception as e:
            print(f"Error finding notes: {e}")
            return [], [], []
        
    def find_longevity(self):
        try:
            longevity = self.driver.find_elements(By.CLASS_NAME, "cell.small-12.medium-6")
            longevity = longevity[4].text
            # print(self.driver.find_elements(By.CLASS_NAME, "cell.small-12.medium-6").get_attribute('outerHTML'))
            lines = longevity.split('\n')[2:]
            result = "; ".join(f"{lines[i]}: {lines[i+1]}" for i in range(0, len(lines), 2))
            return result
        except:
            return None

    def find_sillage(self):
        # all_elements= driver.find_elements(By.CLASS_NAME, "cell.small-12.medium-6")
        # sillage = all_elements[5].text
        # lines = sillage.split('\n')[2:]
        # result = "; ".join(f"{lines[i]}: {lines[i+1]}" for i in range(0, len(lines), 2))
        try:
            sillage = self.driver.find_elements(By.CLASS_NAME, "cell.small-12.medium-6")
            sillage = sillage[5].text
            lines = sillage.split('\n')[2:]
            result = "; ".join(f"{lines[i]}: {lines[i+1]}" for i in range(0, len(lines), 2))
            return result
        except:
            return None
        
    def find_price(self):
        try:
            price = self.driver.find_elements(By.CLASS_NAME, "cell.small-12.medium-6")
            price = price[7].text
            lines = price.split('\n')[2:]
            result = "; ".join(f"{lines[i]}: {lines[i+1]}" for i in range(0, len(lines), 2))
            return result
        except:
            return None

    def scrap_perfume(self, url):
        try:
            self.driver.get(url)
            self.wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))  # Wait for the full page to load

            print("Scraping:", url)  # Check if URL is accessed
            print("Page title:", self.driver.title)  # Check if page loaded

            gender = self.find_gender()
            accords = self.find_accords()
            description = self.find_description()
            pros_and_cons = self.find_pros_and_cons()
            notes = self.find_notes()
            longevity = self.find_longevity()
            sillage = self.find_sillage()
            price = self.find_price()

            return {
                'gender': gender,
                'accords': accords,
                'description': description,
                'pros_and_cons': pros_and_cons,
                'notes': notes,
                'longevity': longevity,
                'sillage': sillage,
                'price': price,
            }

        except Exception as e:
            print(f"Error scraping perfume {url}: {e}")
            return None
        
    def click_show_more(self, how_many_reloads):
        for _ in range(how_many_reloads):
            try:
                # Find the button
                button = self.wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Show more results')]")))
                button.click()  # Click it
                print("Clicked 'Show more results' button")
                
                # Wait for new content to load
                self.wait.until(EC.presence_of_element_located((By.CLASS_NAME, "grid-x")))  
            except:
                print("No more 'Show more results' button found")
                continue  # Stop when button is gone

    def __del__(self):
        if hasattr(self, 'driver'):
            try:
                self.driver.quit()
            except:
                pass

RUN CODE FROM HERE

In [ ]:
driver = Driver()

### CHANGE ONLY THESE ARGUMENTS

# First argument - URL with perfumes list, Second argument - how many clicks to show more perfumes (each click is around 30 more perfumes loaded to the url, 
# but the request will be full after 50 unique perfumes scrapped)  
perfume_list = driver.scrap_list("https://www.fragrantica.com/search/?dizajner=Versace", 10)
# perfume_list = perfume_list[64:]  ### CUT LIST OF PERFUMES IF ALREADY ARE SCRAPPED

### 

for perfume in perfume_list: # Iterate through every perfume
    try:
        perfume_data = driver.scrap_perfume(perfume[1]) # Scrap new URL
    except:
        pass
    finally:
        try: # Sometimes name and producer is not retrieved from request 
            name = perfume[0][0]
            producer = perfume[0][1]
        except:
            name = perfume[0][0]
            producer = '' 
        try:
            row = [
                name, # Name
                producer, # Producer,
                perfume[1], # URL
                perfume_data['gender'], # Gender
                "; ".join([f"{accord[0]} ({accord[1]})" for accord in perfume_data['accords']]),  # Accords
                perfume_data['description'],
                perfume_data['pros_and_cons'],  # Using " | " as a separator for better readability
                perfume_data['notes'][0],  # Top notes (comma-separated)
                perfume_data['notes'][1],  # Middle notes
                perfume_data['notes'][2],   # Base notes
                perfume_data['longevity'],
                perfume_data['sillage'],
                perfume_data['price'],
            ]
        except:
            continue
        with open('fragrance_data.csv', 'a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(row)

No more 'Show more results' button found
No more 'Show more results' button found
No more 'Show more results' button found
No more 'Show more results' button found
No more 'Show more results' button found
